In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import xgboost as xgb
import pickle
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

/Users/peggylee/anaconda3/envs/Peggy/lib/python3.12/site-packages/threadpoolctl.py:1186: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/peggylee/anaconda3/envs/Peggy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/Users/peggylee/Desktop/python/Kaggle_comptete/02_Data'
with open(f"{path}/Temp/reshaped_train_dict.pkl", 'rb') as f:
    loaded_reshaped_train_dict = pickle.load(f)

In [3]:
def create_lag_features(df, lag=3):
    # 創建滾動特徵
    for i in range(1, lag+1):
        df[f'bg_lag_{i}'] = df['bg_value'].shift(i) 
        df[f'insulin_lag_{i}'] =  df['insulin_value'].shift(i) 
        df[f'hr_lag_{i}'] =  df['hr_value'].shift(i) 
        df[f'cals_lag_{i}'] = df['cals_value'].shift(i)
    return df

def create_roll_mean(df):
    ## bg ##
    df['bg_10min'] = df['bg_value'].rolling(window=2, min_periods=1).mean()
    df['bg_20min'] = df['bg_value'].rolling(window=4, min_periods=1).mean()
    df['bg_30min'] = df['bg_value'].rolling(window=6, min_periods=1).mean()
    df['bg_40min'] = df['bg_value'].rolling(window=8, min_periods=1).mean()
    df['bg_50min'] = df['bg_value'].rolling(window=10, min_periods=1).mean()
    df['bg_60min'] = df['bg_value'].rolling(window=12, min_periods=1).mean()
    df['bg_120min'] = df['bg_value'].rolling(window=24, min_periods=1).mean()
    df['bg_180min'] = df['bg_value'].rolling(window=36, min_periods=1).mean()
    df['bg_240min'] = df['bg_value'].rolling(window=48, min_periods=1).mean()
    df['bg_300min'] = df['bg_value'].rolling(window=60, min_periods=1).mean()
    
    ## insulin ##
    df['in_10min'] = df['insulin_value'].rolling(window=2, min_periods=1).mean()
    df['in_20min'] = df['insulin_value'].rolling(window=4, min_periods=1).mean()
    df['in_30min'] = df['insulin_value'].rolling(window=6, min_periods=1).mean()
    df['in_40min'] = df['insulin_value'].rolling(window=8, min_periods=1).mean()
    df['in_50min'] = df['insulin_value'].rolling(window=10, min_periods=1).mean()
    df['in_60min'] = df['insulin_value'].rolling(window=12, min_periods=1).mean()
    df['in_120min'] = df['insulin_value'].rolling(window=24, min_periods=1).mean()
    df['in_180min'] = df['insulin_value'].rolling(window=36, min_periods=1).mean()
    df['in_240min'] = df['insulin_value'].rolling(window=48, min_periods=1).mean()
    df['in_300min'] = df['insulin_value'].rolling(window=60, min_periods=1).mean()
    
    return df

In [4]:
train_imputed = {}

for key, sub_dict in loaded_reshaped_train_dict.items():
    if "train" in sub_dict:
        temp_df = sub_dict['train']
        temp_df = temp_df.drop(columns = ['activity_value','step_value','carbs_value'])
        temp_df = temp_df.fillna(method = 'ffill').fillna(method = 'bfill')
        temp_df = create_lag_features(temp_df, lag = 71)
        temp_df = create_roll_mean(temp_df)
        temp_df = temp_df.drop(columns = ['insulin_value','hr_value','cals_value','day','time_seq'])
        temp_df['target'] =  temp_df['bg_value'].shift(-12)
        temp_df = temp_df.fillna(method = 'ffill').fillna(method = 'bfill')
        train_imputed[key] =  temp_df
combined = pd.concat(train_imputed.values(),ignore_index = True)

In [5]:
train_X = combined.drop(columns = ['target'])
train_Y = combined.target
print("train_X: ", train_X.shape)
print("train_Y: ", train_Y.shape)
X_train, X_valid, Y_train,Y_valid = train_test_split(train_X,train_Y, test_size = 0.2,random_state = 2024)

train_X:  (235366, 305)
train_Y:  (235366,)


In [6]:
def objective(trial):
    """
    A function to train model by using different hyperparameters combinations by Optuna.
    """
    
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'eta': trial.suggest_loguniform('eta', 0.005, 0.3),  # learning rate
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
        'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.09, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 3, 10),
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 10),
        'tree_method': 'hist'
    }
    reg = xgb.XGBRegressor(**params)
    reg.fit(X_train,Y_train,
           eval_set = [(X_valid,Y_valid)],
           verbose = False)
    y_pred_valid = reg.predict(X_valid)
    rmse = mean_squared_error(Y_valid, y_pred_valid, squared=False)
    return rmse
    

In [7]:
%%time
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 10,timeout=900)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)
best_params = study.best_params

[I 2024-11-11 19:24:47,176] A new study created in memory with name: no-name-8d142721-5a07-45a0-99ae-5715a21f5c29
[I 2024-11-11 19:30:46,254] Trial 0 finished with value: 1.242729914353334 and parameters: {'n_estimators': 648, 'eta': 0.06135738282200275, 'max_depth': 10, 'min_child_weight': 0.020631967244899203, 'subsample': 0.636811078502748, 'colsample_bytree': 0.5335867429195772, 'lambda': 5.655217363720725, 'alpha': 0.06174053906799611}. Best is trial 0 with value: 1.242729914353334.
[I 2024-11-11 19:36:29,773] Trial 1 finished with value: 1.2149684954518472 and parameters: {'n_estimators': 937, 'eta': 0.215539393754515, 'max_depth': 8, 'min_child_weight': 0.024176474994718023, 'subsample': 0.7328847827564059, 'colsample_bytree': 0.9537348353876512, 'lambda': 6.552116048730838, 'alpha': 0.022764170153040178}. Best is trial 1 with value: 1.2149684954518472.
[I 2024-11-11 19:42:35,992] Trial 2 finished with value: 1.836884826943438 and parameters: {'n_estimators': 1215, 'eta': 0.0062

Number of finished trials: 3
Best trial parameters: {'n_estimators': 937, 'eta': 0.215539393754515, 'max_depth': 8, 'min_child_weight': 0.024176474994718023, 'subsample': 0.7328847827564059, 'colsample_bytree': 0.9537348353876512, 'lambda': 6.552116048730838, 'alpha': 0.022764170153040178}
Best score: 1.2149684954518472
CPU times: user 1h 4min 42s, sys: 34.7 s, total: 1h 5min 17s
Wall time: 17min 48s


In [8]:
best_params = study.best_params
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_train, Y_train, 
                eval_set=[(X_valid, Y_valid)], 
                eval_metric='rmse', 
                verbose=False)

XGBRegressor(alpha=0.022764170153040178, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9537348353876512, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eta=0.215539393754515, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=6.552116048730838,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=0.024176474994718023,
             missing=nan, monotone_constraints=None, multi_strategy=None,
             n_estimators=937, ...)

In [9]:
## test data ##
with open(f"{path}/Temp/reshaped_test_dict.pkl", 'rb') as f:
    loaded_reshaped_test_dict = pickle.load(f)
    
test_imputed = {}
test_outcome = {}
for key, sub_dict in loaded_reshaped_test_dict.items():
    if "test" in sub_dict:
        temp_df = sub_dict['test']
        temp_df = temp_df.drop(columns = ['activity_value','step_value','carbs_value'])
        temp_df = temp_df.fillna(method = 'ffill').fillna(method = 'bfill')
        temp_df = create_lag_features(temp_df, lag = 71)
        temp_df = create_roll_mean(temp_df)
        temp_df = temp_df.drop(columns = ['insulin_value','hr_value','cals_value'])
        temp_df = temp_df.fillna(method = 'ffill').fillna(method = 'bfill')
        clean_key = key.replace("_test","")
        temp_df['p_num'] = clean_key
        test_imputed[key] =  temp_df
    if 'output' in sub_dict:
        temp = sub_dict['output']
        temp1 = temp.rename(columns = {
            'time_pre_outcome' : 'time_seq',
            'day_outcome' :'day'
            })
        clean_key = key.replace("_test","")
        temp1['p_num'] = clean_key
        test_outcome[key] = temp1
   

In [10]:
combined_test = pd.concat(test_imputed.values(),ignore_index = True)
combined_test_time = combined_test[['day','time_seq','p_num']]
combined_test = combined_test.drop(columns = ['day','time_seq','p_num'])
test_outcome_df = pd.concat(test_outcome.values(),ignore_index = True)


In [11]:
y_pred = final_model.predict(combined_test)

In [12]:
combined_test_pred_df = pd.DataFrame(
    {'pred' : y_pred,
     'p_num': combined_test_time['p_num'],
     'day' : combined_test_time['day'],
     'time_seq' : combined_test_time['time_seq']})

In [13]:
combined_test_pred_df[['day']] = combined_test_pred_df[['day']].astype(int).astype('object')
combined_test_pred_df[['time_seq']] = combined_test_pred_df[['time_seq']].astype('object')
test_outcome_df['time_seq'] = test_outcome_df['time_seq'].astype(str).str.split(' ').str[-1]
test_outcome_df[['day', 'time_seq']] = test_outcome_df[['day', 'time_seq']].astype('object')
test_final = pd.merge(combined_test_pred_df,test_outcome_df, on = ['day', 'time_seq','p_num'],how = 'inner')
test_final = test_final[['id','pred']].rename(columns = {
    'pred' : 'bg+1:00'
})

In [14]:
test_final.to_csv("/Users/peggylee/Desktop/python/Kaggle_comptete/02_Data/temp/submission.csv", index=False)